In [1]:
import megengine as mge
import megengine.functional as F
import numpy as np


import megengine.module as M

In [2]:
def two_layer_conv(x):
    conv_weight = mge.Parameter(np.random.randn(8, 3, 3, 3).astype(np.float32))
    conv_bias = mge.Parameter(np.zeros((1, 8, 1, 1), dtype=np.float32))
    x = F.conv2d(x, conv_weight, conv_bias)
    x = F.relu(x)
    conv_weight = mge.Parameter(np.random.randn(16, 8, 3, 3).astype(np.float32))
    conv_bias = mge.Parameter(np.zeros((1, 16, 1, 1), dtype=np.float32))
    x = F.conv2d(x, conv_weight, conv_bias)
    x = F.relu(x)
    return x

x = mge.tensor(np.random.randn(2, 3, 32, 32).astype(np.float32))
out = two_layer_conv(x)
print(out.shape)

25 23:22:56[mgb] WRN cuda unavailable: CUDA driver version is insufficient for CUDA runtime version(35) ndev=-1


(2, 16, 28, 28)


In [3]:

class ConvReLU(M.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
       
        self.conv_weight = mge.Parameter(np.random.randn(out_channels, in_channels, 3, 3).astype(np.float32))
        self.conv_bias = mge.Parameter(np.zeros((1, out_channels, 1, 1), dtype=np.float32))
        self.relu = M.ReLU()

    def forward(self, x):
        x = F.conv2d(x, self.conv_weight, self.conv_bias)
        x = self.relu(x)
        return x

class TwoLayerConv(M.Module):
    def __init__(self):
        super().__init__()
        self.conv_relu1 = ConvReLU(3, 8)
        self.conv_relu2 = ConvReLU(8, 16)

    def forward(self, x):
        x = self.conv_relu1(x)
        x = self.conv_relu2(x)
        return x

x = mge.tensor(np.random.randn(2, 3, 32, 32).astype(np.float32))    
two_layer_conv_module = TwoLayerConv()
out = two_layer_conv_module(x)
print(out.shape)

(2, 16, 28, 28)


### 实测动静态图转换的时候也很方便，主要是用 jit 包中的 trace 装饰器

In [4]:
from megengine.jit import trace

@trace
def train_func(data, label, *, opt, net): 
    logits = net(data)
    loss = F.cross_entropy_with_softmax(logits, label)
    opt.backward(loss)
    return logits, loss


In [9]:
class Net(M.Module):

    def __init__(self):
        self.fc0 = M.Linear(2, 100)

    def forward(self, inp):
        return self.fc0(inp)

net = Net()
'''
for i in range(100000):
    data = mge.tensor(
        np.random.rand(64, 2).astype(np.float32) * 2 - 1
    )
    net(data)
'''
data = mge.tensor(
np.random.rand(64, 2).astype(np.float32)
)
print(type(data))
print(data.shape)
for i in range(10):
    data.set_value(np.random.rand(64, 2) * 2 -1)
    # data.set_value(data * 2 - 1)
    net(data)

<class 'megengine.core.tensor.Tensor'>
(64, 2)


In [10]:
print(data)

Tensor([[ 0.927   0.1042]
 [ 0.8835  0.0697]
 [-0.4544  0.4407]
 [-0.8283 -0.6549]
 [-0.9535  0.3501]
 [ 0.9772 -0.7974]
 [ 0.536   0.3853]
 [-0.096   0.2484]
 [ 0.6059 -0.4675]
 [-0.319  -0.0927]
 [-0.2578  0.5769]
 [ 0.6835 -0.3111]
 [-0.0559 -0.598 ]
 [-0.6625 -0.1798]
 [ 0.0304  0.9723]
 [-0.8999  0.4183]
 [-0.5565  0.7022]
 [ 0.7117 -0.3448]
 [-0.8451  0.3555]
 [ 0.7735 -0.8291]
 [-0.501   0.9821]
 [ 0.6805 -0.7913]
 [-0.1598  0.8722]
 [-0.5516  0.7803]
 [-0.     -0.6572]
 [ 0.3028 -0.3561]
 [ 0.447  -0.3233]
 [-0.6793  0.6764]
 [ 0.6094  0.9067]
 [-0.5335 -0.6463]
 [-0.6327 -0.5482]
 [ 0.3676  0.3635]
 [ 0.3269 -0.6905]
 [-0.8356  0.7812]
 [ 0.2038 -0.1672]
 [ 0.5433  0.7714]
 [-0.9011 -0.9449]
 [ 0.6092 -0.7962]
 [-0.5369 -0.004 ]
 [ 0.6117 -0.5627]
 [ 0.2563  0.2303]
 [-0.0146  0.5058]
 [ 0.1429  0.1812]
 [ 0.9388 -0.829 ]
 [ 0.3355  0.4483]
 [-0.6645 -0.4136]
 [-0.1015  0.4516]
 [ 0.4088  0.877 ]
 [-0.2481 -0.846 ]
 [-0.1691 -0.0581]
 [ 0.388  -0.1029]
 [-0.1442 -0.0866]
 [-0.